In [1]:
# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql

In [2]:
# Postgres info to connect

connection_args = {
  "host": "twitchdata.chd4n5ul8muk.us-east-2.rds.amazonaws.com",
  "user": "USER",
    "password":"PASSWORD",
  "port": 5432,
  "dbname": "twitchdata"
}


connection = pg.connect(**connection_args)

In [3]:
# Final query after the database collection was stopped. This data was collected over the course of the week from 6/01 through 6/07.
query = "SELECT distinct(time_logged) FROM stream_data order by time_logged desc;"

pd_sql.read_sql(query, connection)

,time_logged
0,2019-06-07 03:02:52
1,2019-06-06 22:02:54
2,2019-06-06 21:02:55
3,2019-06-06 20:02:57
4,2019-06-06 19:04:09
5,2019-06-06 18:03:02
6,2019-06-06 17:04:01
7,2019-06-06 16:02:57
8,2019-06-06 15:03:54
9,2019-06-06 14:04:18


In [6]:
query = "SELECT COUNT(DISTINCT(stream_id)) FROM stream_data;"

pd_sql.read_sql(query, connection)

,count
0,350602


In [4]:
query = "SELECT COUNT(DISTINCT(game_id)) FROM game_information;"

pd_sql.read_sql(query, connection)

,count
0,532


In [5]:
query = "SELECT COUNT(DISTINCT(game_name)) FROM game_genres;"

pd_sql.read_sql(query, connection)

,count
0,1001


In [7]:
query = "SELECT Count(*) FROM stream_data;"

pd_sql.read_sql(query, connection)

,count
0,865196


### Querying the complete dataset for EDA as the new data comes in throughout the day. This query also created the general_view which we proceed to query from for all recommender workbooks. ###

In [8]:
query = "SELECT Count(*) FROM general_view;"

pd_sql.read_sql(query, connection)

,count
0,2954187


In [9]:
query = "SELECT * FROM general_view LIMIT 3;"

pd_sql.read_sql(query, connection)

,stream_id,user_name,game_id,stream_type,title,viewer_count,started_at,language,time_logged,game_name,pic_url,game_genres
0,34366704544,gtzzzz,19619,live,THE REVENGE (http://fearless.nostalrius.com.br...,20,2019-06-01 20:02:46,pt,2019-06-01 23:03:24,Tibia,https://static-cdn.jtvnw.net/ttv-boxart/Tibia-...,MMORPG
1,34366704544,gtzzzz,19619,live,THE REVENGE (http://fearless.nostalrius.com.br...,20,2019-06-01 20:02:46,pt,2019-06-01 23:03:24,Tibia,https://static-cdn.jtvnw.net/ttv-boxart/Tibia-...,Adventure Game
2,34366704544,gtzzzz,19619,live,THE REVENGE (http://fearless.nostalrius.com.br...,20,2019-06-01 20:02:46,pt,2019-06-01 23:03:24,Tibia,https://static-cdn.jtvnw.net/ttv-boxart/Tibia-...,RPG


In [10]:
query = '''SELECT stream_id, user_id, user_name, stream_data.game_id, stream_type, title, viewer_count, started_at, language, time_logged, game_information.game_name, pic_url, game_genres.game_genres 
FROM stream_data 
LEFT JOIN game_information ON stream_data.game_id = game_information.game_id
LEFT JOIN game_genres ON game_information.game_name = game_genres.game_name;'''

complete_data = pd_sql.read_sql(query, connection)

In [12]:
complete_data.to_csv('final_general_view_06_07_19.csv', index = False)

### Getting the data for surprise recommender in the format to be funneled into the Surprise notebook. ###

In [14]:
query = '''SELECT user_name, game_name, game_genres, language, started_at, viewer_count, max(viewer_count)
FROM general_view
GROUP BY user_name, game_name, game_genres, language, started_at, viewer_count
ORDER BY user_name DESC, max DESC'''

surprise_grid = pd_sql.read_sql(query, connection)

In [13]:
surprise_grid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1326955 entries, 0 to 1326954
Data columns (total 7 columns):
user_name       1326955 non-null object
game_name       1325975 non-null object
game_genres     1323776 non-null object
language        1326955 non-null object
started_at      1326955 non-null datetime64[ns]
viewer_count    1326955 non-null int64
max             1326955 non-null int64
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 70.9+ MB


In [15]:
surprise_grid.to_csv('final_game_user_grid_06_07_19.csv', index = False)

In [19]:
# Alternative view of the data used in the early iterations of the recommender model.

In [17]:
query = '''SELECT user_name, game_name, max(viewer_count)
FROM general_view
GROUP BY user_name, game_name
ORDER BY user_name DESC, max DESC'''

minimal_grid = pd_sql.read_sql(query, connection)

In [18]:
minimal_grid.to_csv('final_minimal_grid_06_07_19.csv', index = False)